In [1]:
import twint

In [56]:
# d.Search = "Stocks"
d = twint.Config()
d.Username = "beaker"
d.Limit = 50
# d.Store_csv = True
d.Store_object = True
d.Store_json = True
d.Output = "./test1.json"
# d.Database = "tweets.db"


x = twint.run.Search(d)

1218694727782031360 2020-01-18 16:40:40 Pacific Standard Time <Beaker> LOL. Might be dangerous
1217602526473850881 2020-01-15 16:20:38 Pacific Standard Time <Beaker> Thread https://twitter.com/darkquassar/status/1217253393955049472 …
1217600363832004608 2020-01-15 16:12:03 Pacific Standard Time <Beaker> The CISA call yesterday focused heavily on this
1217581255983882241 2020-01-15 14:56:07 Pacific Standard Time <Beaker> Hysterical https://twitter.com/deviantollam/status/1217569453942046720 …
1217492872154447873 2020-01-15 09:04:55 Pacific Standard Time <Beaker> Indeed https://twitter.com/operant/status/1217430670559121412 …
1217492725311922176 2020-01-15 09:04:20 Pacific Standard Time <Beaker> Yasssssss
1217419766719307777 2020-01-15 04:14:25 Pacific Standard Time <Beaker> Happy anniversary, Amanda. You’re amazing and they are lucky to have you!
1217272490646540289 2020-01-14 18:29:12 Pacific Standard Time <Beaker> Me, either
1217272099800305664 2020-01-14 18:27:39 Pacific Standard Tim

In [42]:
# x = twint.run.Search(d)
# twint -u noneprivacy --csv --output "/Users/psingh4/harsh/test4.json" --lang en --translate --translate-dest it --limit 100

In [43]:
# d.__dict__.keys()

In [44]:
# twint.run.Search(d)
# twint.output

In [64]:
f= open("./test1.json","r",errors='ignore')
line1 = f.readlines()
arr = []
for x in line1:
    arr.append(x)
f.close()    

In [65]:
arr.insert(0,"[")

In [66]:
arr.append(']')
for i in range(1,len(arr)-2):
    arr[i] = arr[i][:-1] + "," + arr[i][-1:]

In [67]:
file = open("./test1.json","w")
for i in range(len(arr)):
    file.write(arr[i])
file.close()

In [69]:
import json
# read_file = "/Users/psingh4/harsh/test.json"
read_file = ""
with open("./test1.json", "r") as read_file:
    data = json.load(read_file)

In [71]:
len(data)

2000

In [122]:
import mysql.connector

In [123]:
mydb = mysql.connector.connect(user='admin', password='Private2712!',
                              host='database-1.cok63qqiofsd.us-east-1.rds.amazonaws.com',
                              database='data')

In [124]:
mycursor = mydb.cursor()

In [125]:
sql = """INSERT INTO train_data (created_at, conversation_id, id, date, time, timezone, user_id, username, name, place, tweet, mentions, urls, replies_count, 
        retweets_count, likes_count, hashtags, cashtags, link, retweet, video, near, geo, source, user_rt_id, user_rt, retweet_id,
        retweet_date, translate, trans_src, trans_dest, photos, reply_to) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

val = []
for i in range(len(data)):
    val.append((data[i]['created_at'], data[i]['conversation_id'], str(data[i]['id']), data[i]['date'], data[i]['time'], data[i]['timezone'], data[i]['user_id'], data[i]['username'],
      data[i]['name'], data[i]['place'], data[i]['tweet'], str(data[i]['mentions']), str(data[i]['urls']), data[i]['replies_count'],
      data[i]['retweets_count'], data[i]['likes_count'], str(data[i]['hashtags']), str(data[i]['cashtags']), data[i]['link'], data[i]['retweet'],
      data[i]['video'], data[i]['near'], data[i]['geo'], data[i]['source'], data[i]['user_rt_id'], data[i]['user_rt'],
      data[i]['retweet_id'], data[i]['retweet_date'], data[i]['translate'], data[i]['trans_src'], data[i]['trans_dest'],str(data[i]['photos']), 
       str(data[i]['reply_to'])))
    
mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "record inserted.")

2000 record inserted.


In [126]:
mydb.close()

In [ ]:
val = []
for i in range(len(data)):
    val.append((data[i]['created_at'], data[i]['conversation_id'], data[i]['date'], data[i]['time'], data[i]['timezone'], data[i]['user_id'], data[i]['username'],
      data[i]['name'], data[i]['place'], data[i]['tweet'], str(data[i]['mentions']), str(data[i]['urls']), data[i]['replies_count'],
      data[i]['retweets_count'], data[i]['likes_count'], str(data[i]['hashtags']), str(data[i]['cashtags']), data[i]['link'], data[i]['retweet'],
      data[i]['video'], data[i]['near'], data[i]['geo'], data[i]['source'], data[i]['user_rt_id'], data[i]['user_rt'],
      data[i]['retweet_id'], data[i]['retweet_date'], data[i]['translate'], data[i]['trans_src'], data[i]['trans_dest'],0,str(data[i]['photos']), 
       str(data[i]['reply_to'])))

In [ ]:
# mydb.commit()
# !pip install tensorflow
# print(mycursor.rowcount, "record inserted.")

In [ ]:
mycursor.execute("SELECT * FROM information_schema.COLUMNS  WHERE TABLE_SCHEMA='data'")

In [ ]:
myresult = mycursor.fetchone()
print(myresult)

In [ ]:
created_at date time timezone user_id username name place tweet mentions urls replies_count 
retweets_count likes_count hastags cashtags link retweet video near geo source user_rt_id user_rt retweet_id
retweet_date translate trans_src trans_dest relevant

In [ ]:
print(data[0]['created_at'], data[0]['date'], data[0]['time'], data[0]['timezone'], data[0]['user_id'], data[0]['username'],
      data[0]['name'], data[0]['place'], data[0]['tweet'], str(data[0]['mentions']), str(data[0]['urls']), data[0]['replies_count'],
      data[0]['retweets_count'], data[0]['likes_count'], str(data[0]['hashtags']), str(data[0]['cashtags']), data[0]['link'], data[0]['retweet'],
      data[0]['video'], data[0]['near'], data[0]['geo'], data[0]['source'], data[0]['user_rt_id'], data[0]['user_rt'],
      data[0]['retweet_id'], data[0]['retweet_date'], data[0]['translate'], data[0]['trans_src'], data[0]['trans_dest'],0,str(data[0]['photos']), 
       str(data[0]['reply_to']))

In [ ]:
import datetime
x = datetime.time(17,59,51)
print(x.date())
t = datetime.time(1, 2, 3)
print(t)
x = datetime.datetime(2020,1,1,17,2,3)
print(x.date().isoformat())
import time    
time.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# !pip install bert-serving-server
# !pip install bert-serving-client

In [111]:
data[0]

{'cashtags': [],
 'conversation_id': '1218605167823806464',
 'created_at': 1579387325000,
 'date': '2020-01-18',
 'geo': '',
 'hashtags': [],
 'id': 1218664887381041155,
 'likes_count': 4,
 'link': 'https://twitter.com/briankrebs/status/1218664887381041155',
 'mentions': ['thewrongphil', 'redteamwrangler'],
 'name': 'briankrebs',
 'near': '',
 'photos': [],
 'place': '',
 'quote_url': '',
 'replies_count': 0,
 'reply_to': [{'user_id': '22790881', 'username': 'briankrebs'},
  {'user_id': '3560292449', 'username': 'thewrongphil'},
  {'user_id': '200950703', 'username': 'redteamwrangler'}],
 'retweet': False,
 'retweet_date': '',
 'retweet_id': '',
 'retweets_count': 0,
 'source': '',
 'time': '14:42:05',
 'timezone': 'Pacific Standard Time',
 'trans_dest': '',
 'trans_src': '',
 'translate': '',
 'tweet': 'You will soon....',
 'urls': [],
 'user_id': 22790881,
 'user_rt': '',
 'user_rt_id': '',
 'username': 'briankrebs',
 'video': 0}

In [78]:
data[7]['conversation_id']

'1217152211056238593'

In [91]:
val[0]

(1579387325000,
 '1218605167823806464',
 '2020-01-18',
 '14:42:05',
 'Pacific Standard Time',
 22790881,
 'briankrebs',
 'briankrebs',
 '',
 'You will soon....',
 "['thewrongphil', 'redteamwrangler']",
 '[]',
 0,
 0,
 4,
 '[]',
 '[]',
 'https://twitter.com/briankrebs/status/1218664887381041155',
 False,
 0,
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '[]',
 "[{'user_id': '22790881', 'username': 'briankrebs'}, {'user_id': '3560292449', 'username': 'thewrongphil'}, {'user_id': '200950703', 'username': 'redteamwrangler'}]")

In [2]:
!pip install neo4j

  Created wheel for neo4j: filename=neo4j-1.7.6-py3-none-any.whl size=34347 sha256=046ff7b2b3f9c73074df8f277da7c5d6da26045f83ac8e80dcc073dd0a2dfcde
  Stored in directory: c:\users\harsh\appdata\local\pip\cache\wheels\d7\1a\55\e3e4e254143635c20053f8f39301e1f58a79a07d75d6286c02
  Created wheel for neobolt: filename=neobolt-1.7.16-py3-none-any.whl size=38153 sha256=f48754433a290e979bff72240be1232c9542b56717c4e8e782be53c60e353a50
  Stored in directory: c:\users\harsh\appdata\local\pip\cache\wheels\98\63\31\98f742c7a55528a7bd87e273b48dbfdebe4d19b9c2bd4f32da
  Created wheel for neotime: filename=neotime-1.7.4-py3-none-any.whl size=17779 sha256=f2ac0aa6aa08a3e329f767785a7652a99a702d21e65e86542aa568a89ec011d7
  Stored in directory: c:\users\harsh\appdata\local\pip\cache\wheels\2f\a5\75\e4c94412ac76860c05f9042cfa656900f3635903dee6696629
Successfully built neo4j neobolt neotime


You should consider upgrading via the 'c:\users\harsh\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
from neo4j import GraphDatabase
import json
#Indicate to driver where neo4j is running
driver = GraphDatabase.driver("bolt://localhost",auth=("test_user","password"))
#Parse data
with open("./data.json",'r') as file:
    #Need to make a session where you will run all your cypher queries
    with driver.session() as session:
        count = 0
        tx = session.begin_transaction()
        for line in file.readlines(): #can limit lines with [:100] after ()
            if (line == "[\n" or line == "]"):
                continue
            item = json.loads(line[:-2])
            # print(item)
            '''WITH {tweet} AS Tweet
                Merge (a:Tweet{id:$value.id,date:value.date,train_id:value.train_id})
                Merge (d:Date{date:value.date})
                Merge (a)-[:SAME_DATE]->(d)'''
            '''CREATE (a:Tweet{id:$value.id,date:$value.date,train_id:$value.train_id})'''
            tx.run('''start n=node(*) return n''',
                   parameters={'tweet': item}, value=item) #add paramets here
            #Batch processing to run 1000 tweets as a time as these commits are quite time intensive
            count += 1
            tx.commit()
            break
            if count > 1000:
                tx.commit()
                tx = session.begin_transaction()
                count = 0
#         tx.commit()

ServiceUnavailable: Failed to establish connection to ('127.0.0.1', 7687) (reason [WinError 10061] No connection could be made because the target machine actively refused it)

In [49]:
import json
with open("./data.json", "r") as read_file:
    for line in read_file.readlines():
        print(line)
        if (line == "[\n" or line == "]"):
            continue
        item = json.loads(line[:-2])
        print(item)


[

{"id": 1223833177518592000, "conversation_id": "1223833177518592000", "created_at": 1580619541000, "date": "2020-02-01", "time": "20:59:01", "timezone": "Pacific Standard Time", "user_id": 1603741075, "username": "reddit_datasets", "name": "/r/datasets", "place": "", "tweet": "Malware and benign windows PE cuckoo reports  http://dlvr.it/RPCpXg  ", "mentions": [], "urls": ["http://dlvr.it/RPCpXg"], "photos": [], "replies_count": 0, "retweets_count": 0, "likes_count": 0, "hashtags": [], "cashtags": [], "link": "https://twitter.com/reddit_datasets/status/1223833177518592000", "retweet": false, "quote_url": "", "video": 0, "near": "", "geo": "", "source": "", "user_rt_id": "", "user_rt": "", "retweet_id": "", "reply_to": [{"user_id": "1603741075", "username": "reddit_datasets"}], "retweet_date": "", "translate": "", "trans_src": "", "trans_dest": ""},

{'id': 1223833177518592000, 'conversation_id': '1223833177518592000', 'created_at': 1580619541000, 'date': '2020-02-01', 'time': '20:59:

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



JSONDecodeError: Expecting ',' delimiter: line 1 column 1013 (char 1012)

In [60]:
import nltk